<div class="alert alert-success">
    <h2 align='center'>📔 Загрузим, Импортирум, и приготовим данные 👨🏻‍🍳</h2>
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q timm
# pip install -q einops
# pip install -q rich
# pip install -q wandb --upgrade;

!cp -R /content/drive/MyDrive/Olimpiads/NTO_2022/videos/videos /content/
!cp -R /content/drive/MyDrive/Olimpiads/NTO_2022/train_video_class_uni_nobeer_1.csv /content/

In [ ]:
import os
import sys
import re
import gc
import platform
import random

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import io
import PIL
import torchvision
import more_itertools
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import pickle
from tqdm.contrib import tzip
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score

# import einops

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

import timm
import glob

# from rich import print as _pprint
# from rich.progress import track

import albumentations as A
from albumentations.pytorch import ToTensorV2

# import wandb

import warnings
warnings.simplefilter('ignore')

<div class="alert alert-success">
    <h2 align='center'>🚀 Config Dictionary and W&B Integration </h2>
</div>

In [ ]:
Config = dict(
    MAX_FRAMES = 6,
    EPOCHS = 30,
    LR = 2e-4,
    IMG_SIZE = (224, 224),
    FEATURE_EXTRACTOR = 'resnext101_32x8d',
    DR_RATE = 0.35,
    NUM_CLASSES = 9,
    RNN_HIDDEN_SIZE = 256,
    RNN_LAYERS = 1,
    TRAIN_BS = 64,
    VALID_BS = 32,
    NUM_WORKERS = 2
)

In [ ]:
def read_video(path, transform=None, frames_num=4):
    frames = []
    cap = cv2.VideoCapture(path)
    
    fps = int(cap.get(cv2.CAP_PROP_FPS)) # frame rate
    
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Number of frames in the video file
    N = length//(frames_num) 

    if N == 0: print(path)

    current_frame = 1
    for i in range(length):
        ret, frame = cap.read(current_frame)
        
        if ret and i==current_frame and len(frames)<frames_num:
           
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, Config['IMG_SIZE'])
            
            # frames.append(Augments.valid_augments(image=frame)['image'])
            frames.append(frame)
            current_frame += N
        
    cap.release()
    return frames

<div class="alert alert-success">
    <h2 align='center'>💻 Custom Dataset Class</h2>
</div>

In [ ]:
df = pd.read_csv('train_video_class_uni_nobeer_1.csv')
df.columns = ['unnamed', 'video', 'label']
df = df[(df.label != 'cloud, sunset') & (df.label != 'cloud, water') & (df.label != 'water, flower')]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.video, df.label, test_size=0.2, random_state=42, stratify=df.label)
df_train = pd.DataFrame({'path':X_train, 'labels':y_train})
df_test = pd.DataFrame({'path':X_test, 'labels':y_test})

In [ ]:
class RSNADataset(Dataset):
    def __init__(self, df,  is_test):
        self.df = df

        self.train_transform = A.Compose([
            # A.RandomBrightness(limit=0.3, p=.8),
            # A.CLAHE(p=.3),
            # A.ToGray(p=.2),
            # A.GridDistortion(p=.3, border_mode=cv2.BORDER_CONSTANT, num_steps=5, distort_limit=0.3),
            # A.GaussianBlur(blur_limit=(1, 3), always_apply=False, p=0.35),
            # A.RandomCrop(width=190, height=190, p=0.3),
            A.RandomResizedCrop(Config['IMG_SIZE'][0], Config['IMG_SIZE'][1], scale=(0.8, 1)),
            A.Rotate((-7, 7), p=0.2),
            A.HorizontalFlip(p=0.3),
            A.RGBShift(r_shift_limit=5, g_shift_limit=5, b_shift_limit=5, p=0.2),
            A.RandomGamma(gamma_limit=(90, 110), p=0.2),
            A.Blur(blur_limit=3, p=0.1),
            A.OpticalDistortion(distort_limit=0.03, shift_limit=0.03, p=0.1),
            A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=3, val_shift_limit=3, p=0.2),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.4),

            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

            ToTensorV2(p=1.0),
            ], p=1.)
        
        self.test_transform = A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(p=1.0),
            ], p=1.)

        self.is_test = is_test
        self.samples = []
        self.cls = {'water':0,'car':1,'cloud':2,'food':3,'flower':4,'dance':5,'animal':6,'sunset':7,'fire':8}
        
        for path, label in tzip(df.path, df.labels):
          frames = read_video(path=f'videos/{path}', frames_num = Config['MAX_FRAMES'])
          self.samples +=[{'label': self.cls[str(label)],'frames': frames}]
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        frames = sample['frames']

        if self.is_test == False:
            for i in range(len(frames)):
                frames[i] = self.train_transform(image=frames[i])['image']
        elif self.is_test == True:
            for i in range(len(frames)):
                frames[i] = self.test_transform(image=frames[i])['image']

        frames_tr = torch.from_numpy(np.stack(frames, axis=0)) 
            
        label = torch.tensor(sample['label']).long()
        return frames_tr, label

<div class="alert alert-success">
    <h2 align='center'>📈 Model Class with ResNext Backbone</h2>
</div>

In [ ]:
class ResNextModel(nn.Module):
    def __init__(self):
        super(ResNextModel, self).__init__()
        self.backbone =  torchvision.models.resnext101_32x8d(weights='IMAGENET1K_V2') # timm.create_model(Config['FEATURE_EXTRACTOR'], pretrained=True) # torchvision.models.efficientnet_v2_s(weights='IMAGENET1K_V1') # torchvision.models.resnext101_32x8d(weights='IMAGENET1K_V2') torchvision.models.efficientnet_v2_s(weights='IMAGENET1K_V1')
    def forward(self, x):
        return self.backbone(x)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class RSNAModel(nn.Module):
    def __init__(self, pretrained=True):
        super(RSNAModel, self).__init__()
        self.backbone = ResNextModel()
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.num_features = self.backbone.backbone.fc.in_features
        # self.num_features = self.backbone.backbone.classifier[1].in_features
        
        self.backbone.backbone.fc = Identity()
        # self.backbone.backbone.classifier = Identity()
        self.dropout= nn.Dropout(Config['DR_RATE'])
        self.rnn = nn.LSTM(self.num_features, Config['RNN_HIDDEN_SIZE'], Config['RNN_LAYERS'], dropout=0.1, batch_first=True, bidirectional=True )
        self.fc1 = nn.Linear(Config['RNN_HIDDEN_SIZE'] * 2, Config['NUM_CLASSES'])
        
    def forward(self, x):
        b, f, c, h, w = x.shape
        #print(x.reshape(b * f, c, h, w).shape)
        x = self.backbone(x.reshape(b * f, c, h, w))
        #print(x.shape)
        x = x.reshape(b, f, self.num_features)
        #print(x.shape)
        x, _ = self.rnn(x)
        #print(x.shape)
        x = self.dropout(x[:, -1])
        #print(x.shape, self.num_features)
        x = self.fc1(x)
        return x

<div class="alert alert-success">
    <h2 align='center'>🏴‍☠️ Training and Validation Functions</h2>
</div>

In [ ]:
def train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch, device, log_wandb=True, verbose=False):
    """
    Trains model for one epoch
    """
    scaler = torch.cuda.amp.GradScaler()
    model.train()
    running_loss = 0
    prog_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    for batch, (frames, targets) in prog_bar:
        optimizer.zero_grad()
        
        frames = frames.to(device, torch.float)
        targets = targets.to(device)
        
        with torch.cuda.amp.autocast():
        # Re arrange the frames in the format our model wants to recieve
            preds = model(frames)
            loss = loss_fn(preds, targets)
        
        # loss.backward()
        # optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        
        loss_item = loss.item()
        running_loss += loss_item
        
        prog_bar.set_description(f"loss: {loss_item:.4f}")
        
        # if log_wandb == True:
        #     wandb_log(
        #         batch_train_loss=loss_item
        #     )
        # 
        # if verbose == True and batch % 20 == 0:
        #     print(f"Batch: {batch}, Loss: {loss_item}")

        scaler.update()
    
    avg_loss = running_loss / len(train_dataloader)
    
    return avg_loss

@torch.no_grad()
def valid_one_epoch(model, valid_dataloader, loss_fn, epoch, device, log_wandb=True, verbose=False):
    """
    Validates the model for one epoch
    """
    model.eval()
    running_loss = 0
    prog_bar = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader))
    for batch, (frames, targets) in prog_bar:
        frames = frames.to(device, torch.float)
        targets = targets.to(device)
        
        # Re arrange the frames in the format our model wants to recieve
        preds = model(frames)
        loss = loss_fn(preds, targets)
        # print('accuracy:', accuracy_score(targets, preds))
        
        loss_item = loss.item()
        running_loss += loss_item
        
        prog_bar.set_description(f"val_loss: {loss_item:.4f}")
        
        if log_wandb == True:
            wandb_log(
                batch_val_loss=loss_item
            )
        
        if verbose == True and batch % 10 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")
    
    avg_val_loss = running_loss / len(valid_dataloader)
    
    return avg_val_loss

<div class="alert alert-success">
    <h2 align='center'>🏗 Обучим модель</h2>
</div>

In [ ]:
log_wandb = False
if torch.cuda.is_available():
        print("Using GPU: {}\n".format(torch.cuda.get_device_name()))
        device = torch.device('cuda')
else:
        print("\nGPU not found. Using CPU: {}\n".format(platform.processor()))
        device = torch.device('cpu')
    
train_df = df_train # pd.read_csv('train.csv')
valid_df = df_test # pd.read_csv('test.csv')
    
print(f'Size of Training Set: {len(train_df)}, Validation Set: {len(valid_df)}')

train_data = RSNADataset(train_df, is_test=False)
valid_data = RSNADataset(valid_df, is_test=True)
train_loader = DataLoader(
        train_data,
        batch_size=Config['TRAIN_BS'], 
        shuffle=True,
        num_workers=Config['NUM_WORKERS']
)

valid_loader = DataLoader(
        valid_data, 
        batch_size=Config['VALID_BS'], 
        shuffle=False,
        num_workers=Config['NUM_WORKERS']
)

Using GPU: Tesla T4

Size of Training Set: 416, Validation Set: 104


  0%|          | 0/416 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

In [ ]:
model = RSNAModel()
model = model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=Config['LR'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1, patience=3, verbose=True)

train_loss_fn = nn.CrossEntropyLoss()
valid_loss_fn = nn.CrossEntropyLoss()
    
print(f"\nUsing Backbone: {Config['FEATURE_EXTRACTOR']}")


Using Backbone: resnext101_32x8d


In [ ]:
current_loss = 1000
for epoch in range(Config['EPOCHS']):
    print(f"\n{'--'*8} EPOCH: {epoch+1} {'--'*8}\n")
        
    train_loss = train_one_epoch(model, train_loader, optimizer, train_loss_fn, epoch=epoch, device=device, log_wandb=log_wandb)
        
    valid_loss = valid_one_epoch(model, valid_loader, valid_loss_fn, epoch=epoch, device=device, log_wandb=log_wandb)
        
    print(f"val_loss: {valid_loss:.4f}")
    scheduler.step(valid_loss)
        
    # if log_wandb == True:
    #     wandb_log(
    #             train_loss=train_loss,
    #             valid_loss=valid_loss
    #     )
        
    if valid_loss < current_loss:
        current_loss = valid_loss
        torch.save(model.state_dict(), f"/content/drive/MyDrive/Olimpiads/NTO_2022/model_{Config['FEATURE_EXTRACTOR']}.pt")


---------------- EPOCH: 1 ----------------



  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

val_loss: 2.0456

---------------- EPOCH: 2 ----------------



  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

val_loss: 0.5293
Epoch 00015: reducing learning rate of group 0 to 2.0000e-05.

---------------- EPOCH: 16 ----------------



  0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

1024

<div class="alert alert-success">
    <h2 align='center'>🏗 Посчитаем метрики</h2>
</div>

In [ ]:
gt = []
pred = []
model.eval()
prog_bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
for batch, (frames, targets) in prog_bar:
    frames = frames.to('cuda').float()
    gt +=  list(targets.numpy())
    with torch.no_grad():

        pred += list(model(frames).argmax(dim=-1).to('cpu').flatten().numpy())


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score 
print(accuracy_score(gt, pred), f1_score(gt, pred, average='weighted'))

0.8461538461538461 0.847281275164814


In [ ]:
!cp /content/model_resnext101_32x8d.pt /content/drive/MyDrive/Olimpiads/NTO_2022/